In [1]:
display(HTML("<style>.rendered_html.text_cell_render {max-width:700px; }</style>"));

HTML{String}("<style>.rendered_html.text_cell_render {max-width:700px; }</style>")

In [4]:
using Interact

In [11]:
include("src/smc_move.jl")

Loading `data/data_3.jld2` ...

	_segs 	Vector{Segment}
	_xs 	Vector{Vector{Float64}}
	_hds 	Vector{Float64}
	_ps 	Vector{Pose}
	_dxs 	Vector{Vector{Float64}}
	_dhds 	Vector{Float64}
	_us 	Vector{Tuple{Vector{Float64}, Float64}}
	_as 	Vector{Float64}
	_zs 	Vector{Any}
	_ss 	Vector{Vector{Measurement}}
	_fov 	Float64
	_zmax 	Int64
	_T 	Int64

To take a look call `glimpse_at_data(_segs, _ps, _ss)`


constraints (generic function with 1 method)

In [12]:
function naive_posterior(T, N, args)

    ch = choicemap()
    for t=0:T
        ch = merge(ch, constraints(t))
    end
    
    trs, ws, = importance_sampling(static_slam_model, (T,args...), ch, N);
    
    probs = exp.(ws .- logsumexp_cu(ws, 1))
    i = categorical(probs)
    tr = trs[i]
    w  = ws[i]
    
    tr, w
end

naive_posterior (generic function with 1 method)

In [13]:
function particle_filter_posterior(T, N, args)
    
    ch = constraints(0)
    tr, w = generate(static_slam_model, (0,args...), ch);
    
    trs = fill(tr, N)
    ws  = fill(w, N)
    
    diffs = argdiffs([1;fill(0,length(args))])
    
    for t=1:T
        ch = constraints(t)
        
        for i=1:N
            tr, dw, _, _ = Gen.update(trs[i], (t, args...), diffs, ch); 
            trs[i] = tr
            ws[i]  = ws[i] + dw 
        end
        
        # Resampling  
        probs = exp.(ws .- logsumexp_cu(ws, 1))
        inds = Distributions.rand(Distributions.Categorical(probs), (N,))
        trs = trs[inds]
        ws = zeros(N)
    end
    
    probs = exp.(ws .- logsumexp_cu(ws, 1))
    
    i = categorical(probs)
    tr = trs[i]
    w  = ws[i]
    tr, w
end

simple_posterior (generic function with 1 method)

In [14]:
function smc_posterior(T, args, grid_args)


    ch = constraints(0)
        
    tr, w = generate(static_slam_model, (0,args...), ch);
    for t=1:T

        x  = stack(polar_inv(_zs[t+1],_as))
        ch = constraints(t)
        
        # Similar to `SimpleExtendingTraceTranslator` with 
        # an implicit custom grid pose propsal
        tr, dw = extend(tr, ch, grid_args);
        w = w + dw
    end
    return tr, w
end

smc_posterior (generic function with 1 method)

In [15]:
function perturb(us::Vector{Tuple{Vector{Float64}, Float64}}, x_noise=0.1, hd_noise=10.0)
   us_tilde = [(
            u[1]+diagnormal([0;0],[x_noise;x_noise]),
            u[2]+normal(0,hd_noise/360*2π))
        for u in us
    ]
    return us_tilde
end

perturb (generic function with 4 methods)

In [30]:
us_noisy = _us

function resample_us(x_noise, hd_noise)
    global us_noisy;
    us_noisy = perturb(_us, x_noise, hd_noise)
end

resample_us (generic function with 1 method)

In [31]:
p0 = _ps[1]
x0_noise  = .0001
hd0_noise = .0001

# Motion/Control noise
x_noise  = 0.25
hd_noise = 10.

# Sensor args
wind    = 10
s_noise = 0.1
dropout = 1e-8



args = (
    _us,         
    p0, x0_noise, hd0_noise, # Pose prior model
    x_noise, hd_noise,       # Motion model
    wind, s_noise, dropout   # Sensor model
);

get_model_args(x_noise, hd_noise, wind, s_noise) = (
    us_noisy,         
    p0, x0_noise, hd0_noise, # Pose prior model
    x_noise, hd_noise,       # Motion model
    wind, s_noise, dropout   # Sensor model
);

In [32]:
n   = 5
w   = fill(20, n)
sig = fill(0.1,n)
fill(dropout,n)

grid_k   = [4, 4, 4, 4, 4]
grid_dx  = [1., 0.4, 0.125, 0.125, 0.125]
grid_dhd = [90.0, 45, 22.5, 10, 10]

grid_args = (w, sig, dropout, grid_k, grid_dx, grid_dhd);

get_grid_args(n, k, w,sig) = (
    fill(w, n), 
    fill(sig, n), 
    fill(1e-9,n), 
    fill(k, n), 
    collect(range(1.0, 0.125, n)),
    collect(range(90.0, 10., n))
);

In [33]:
args = get_model_args(0.1, 5.0, 10, 0.2)
grid_args = get_grid_args(4, 4, 10, 0.2)

naive_posterior(1, 10, args);
simple_posterior(1, 10, args);
smc_posterior(1, args, grid_args);

## Demo

In [34]:
using Interact

In [35]:
function plot_it(
        posterior, show_u, show_x, show_samples,
        n,T,N, 
        x_noise, hd_noise, 
        mix_w, s_noise, 
        grid_n, grid_k, grid_w, grid_s_noise
        )
    args = get_model_args(x_noise, hd_noise, mix_w, s_noise);
    grid_args = get_grid_args(grid_n, grid_k, grid_w, grid_s_noise);
    
    trs = []
    ws = []
    for i=1:n
        print(i)
        if posterior == :naive
            tr,w = naive_posterior(T, N, args)
        elseif posterior == :pf
            tr,w = particle_filter_posterior(T, N, args)
        elseif posterior == :smc
            tr,w = smc_posterior(T, args, grid_args)
        end
        push!(trs, tr)
        push!(ws, w)
    end
    
    

    ani = Animation()
    for i=1:length(trs)
        
        mypl = plot(
            size=(500,500), legend=nothing, 
            title="$(posterior)\nσₓ = $(x_noise)\nσᵣ = $(hd_noise)"; 
            xlim=(4,17),
            ylim=(8,19),
            aspect_ratio=:equal)

        plot!(_segs, c=:black, linewidth=2)

        if show_x
            tt = T + 1
            x_true = polar_inv(_zs[tt],_as)
            scatter!(x_true.*_ps[tt], c=:red, markersize=2, markerstrokewidth=0, alpha=0.5 )
        end
        
        
        if show_samples
            n=14
            ss = range(0,0.9,n)
            for j=1:n

                s= ss[j]
                i_ = mod(i+j-1,length(trs))+1
                tr = trs[i_]

                ps = [r[1] for r in tr[]];
                xs = [r[2] for r in tr[]];

                c = :lightgray
                plot!([p.x for p in ps], c=c, linewidth=1, alpha=range(0,0.8,n)[j] )
                plot!(ps, c=c, r=0.25, linewidth=2, alpha=range(0,1.,n)[j] )
            end
        end

        plot!(_ps[1:T+1],c=:black, r=0.5, linewidth=2)
        
        if show_x
            tt = T + 1
            plot!([_ps[tt]],c=:red, r=0.5, linewidth=3)
        end
        
        if show_u
            pus = [Pose(p.x+u[1],p.hd+u[2]) for (p,u) in zip(_ps[2:end],args[1])]
            for tt in 1:T
                plot!([_ps[1+tt].x,pus[tt].x],c=:green, r=0.25, linewidth=2, alpha=0.5)
            end
            plot!(pus[1:T],c=:green, r=0.25, linewidth=2)
        end


        frame(ani, mypl)
    end
    
    gif(ani, fps=15)
end

plot_it (generic function with 1 method)

In [41]:
n = slider(1:20:60, value=1, label="# samples")
T = slider(5:20, value=5, label="T")
N = slider(1:50:250, value=1, label="# particles")

show_u = toggle(label = "show u", value=false)
show_x = toggle(label = "show obs", value=false)
show_samples = toggle(label = "show samples", value=true)
posterior = togglebuttons([:naive, :pf, :smc])

x_noise  = slider(range(0.001,0.5,5), label="n=x_noise")
hd_noise = slider(range(0.001,45.,5), label="hd_noise")
mix_w    = slider( 1:5:50, label="mixture size")
s_noise  = slider(range(0.01,1.0,10), label="s_noise")

grid_n = slider(1:10,label="grid iters")
grid_k = slider(2:10,label="grid shape")
grid_w = slider(1:5:50,label="grid mixture size")
grid_s_noise = slider(range(0.01,1.0,10),label="grid s_noise")

sample   = button("Sample and Re-draw")
sample_u = button("Sample Controls")

Interact.@map (&sample_u;  resample_us(x_noise[],hd_noise[]))
plt = Interact.@map (&sample;  plot_it(
        posterior[], show_u[], show_x[], show_samples[],
        n[],T[],N[], 
        x_noise[], hd_noise[], 
        mix_w[], s_noise[], 
        grid_n[], grid_k[], grid_w[], grid_s_noise[]
        )
)

vbox(
    pad(0.1em, sample),
    pad(0.1em, sample_u),
    vbox(posterior),
    vbox(show_u, show_x, show_samples),
    hbox(n,T,N),
    hbox(x_noise, hd_noise), hbox(mix_w, s_noise),
    hbox(grid_n, grid_k), hbox(grid_w, grid_s_noise),
    plt
)

1

[ Info: Saved animation to /home/ubuntu/workspace/probabilistic-slam-in-gen/notebooks/tmp.gif


(div { style=Dict("display" => "flex", "flex-direction" => "column") }
  (div { style=Dict{String, Measures.Length{:em, Float64}}("padding" => 0.1em) }
    Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :button), Any["Sample and Re-draw"], Dict{Symbol, Any}(:attributes => Dict{Any, Any}("data-bind" => "click: function (){this.clicks(this.clicks()+1)}, css: {'is-loading' : loading}"), :className => "is-medium button is-primary", :style => Dict{String, Any}())), Dict{String, Tuple{AbstractObservable, Union{Nothing, Bool}}}("loading" => (Observable(false), nothing), "clicks" => (Observable(0), nothing)), Set{String}(), nothing, Asset[Asset("js", "knockout", "/home/ubuntu/.julia/packages/Knockout/HReiN/src/../assets/knockout.js"), Asset("js", "knockout_punches", "/home/ubuntu/.julia/packages/Knockout/HReiN/src/../assets/knockout_punches.js"), Asset("js", nothing, "/home/ubuntu/.julia/packages/InteractBase/Qhvxg/src/../assets/all.js"), Asset("css", nothing, "/home/ubuntu/.julia/packages/InteractBase/Qhvxg/src/../assets/style.css"), Asset("css", nothing, "/home/ubuntu/.julia/packages/Interact/PENUy/src/../assets/bulma_confined.min.css")], Dict{Any, Any}("loading" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"loading\"]()) ? (this.valueFromJulia[\"loading\"]=true, this.model[\"loading\"](val)) : undefined})")], "clicks" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"clicks\"]()) ? (this.valueFromJulia[\"clicks\"]=true, this.model[\"clicks\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(32), Set{AbstractConnection}(), Condition(Base.InvasiveLinkedList{Task}(Task (runnable) @0x00007f6ac7579b60, Task (runnable) @0x00007f6ac7579b60), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"loading\":WebIO.getval({\"name\":\"loading\",\"scope\":\"10223739802934581850\",\"id\":\"13149978056255441249\",\"type\":\"observable\"}),\"clicks\":WebIO.getval({\"name\":\"clicks\",\"scope\":\"10223739802934581850\",\"id\":\"10786020524047741378\",\"type\":\"observable\"})};\n    var self = this;\n    function AppViewModel() {\n        for (var key in json_data) {\n            var el = json_data[key];\n            this[key] = Array.isArray(el) ? ko.observableArray(el) : ko.observable(el);\n        }\n        \n        \n        [this[\"loading\"].subscribe((function (val){!(this.valueFromJulia[\"loading\"]) ? (WebIO.setval({\"name\":\"loading\",\"scope\":\"10223739802934581850\",\"id\":\"13149978056255441249\",\"type\":\"observable\"},val)) : undefined; return this.valueFromJulia[\"loading\"]=false}),self),this[\"clicks\"].subscribe((function (val){!(this.valueFromJulia[\"clicks\"]) ? (WebIO.setval({\"name\":\"clicks\",\"scope\":\"10223739802934581850\",\"id\":\"10786020524047741378\",\"type\":\"observable\"},val)) : undefined; ret

1

[ Info: Saved animation to /home/ubuntu/workspace/probabilistic-slam-in-gen/notebooks/tmp.gif


...

[ Info: Saved animation to /home/ubuntu/workspace/probabilistic-slam-in-gen/notebooks/tmp.gif


1

[ Info: Saved animation to /home/ubuntu/workspace/probabilistic-slam-in-gen/notebooks/tmp.gif


1234567891011121314151617181920212223242526272829303132333435363738394041

[ Info: Saved animation to /home/ubuntu/workspace/probabilistic-slam-in-gen/notebooks/tmp.gif


## Demo 2

In [ ]:
function plot_it(
        posterior, show_u, show_x, show_samples,
        n,T,N, 
        x_noise, hd_noise, 
        mix_w, s_noise, 
        grid_n, grid_k, grid_w, grid_s_noise
        )
    
    args = get_model_args(x_noise, hd_noise, mix_w, s_noise);
    grid_args = get_grid_args(grid_n, grid_k, grid_w, grid_s_noise);
    
    trs = []
    ws = []
    for i=1:n
        print(i)
        if posterior == :naive
            tr,w = naive_posterior(T, N, args)
        elseif posterior == :pf
            tr,w = particle_filter_posterior(T, N, args)
        elseif posterior == :smc
            tr,w = smc_posterior(T, args, grid_args)
        end
        push!(trs, tr)
        push!(ws, w)
    end
    
    

    ani = Animation()
    for i=1:length(trs)
        
        mypl = plot(
            size=(500,500), legend=nothing, 
            title="$(posterior)\nσₓ = $(x_noise)\nσᵣ = $(hd_noise)"; 
            xlim=(4,17),
            ylim=(8,19),
            aspect_ratio=:equal)

        plot!(_segs, c=:black, linewidth=2)

        if show_x
            tt = T + 1
            x_true = polar_inv(_zs[tt],_as)
            scatter!(x_true.*_ps[tt], c=:red, markersize=2, markerstrokewidth=0, alpha=0.5 )
        end
        
        
        if show_samples
            n=14
            ss = range(0,0.9,n)
            for j=1:n

                s= ss[j]
                i_ = mod(i+j-1,length(trs))+1
                tr = trs[i_]

                ps = [r[1] for r in tr[]];
                xs = [r[2] for r in tr[]];

                c = :lightgray
                plot!([p.x for p in ps], c=c, linewidth=1, alpha=range(0,0.8,n)[j] )
                plot!(ps, c=c, r=0.25, linewidth=2, alpha=range(0,1.,n)[j] )
            end
        end

        plot!(_ps[1:T+1],c=:black, r=0.5, linewidth=2)
        
        if show_x
            tt = T + 1
            plot!([_ps[tt]],c=:red, r=0.5, linewidth=3)
        end
        
        if show_u
            pus = [Pose(p.x+u[1],p.hd+u[2]) for (p,u) in zip(_ps[2:end],args[1])]
            for tt in 1:T
                plot!([_ps[1+tt].x,pus[tt].x],c=:green, r=0.25, linewidth=2, alpha=0.5)
            end
            plot!(pus[1:T],c=:green, r=0.25, linewidth=2)
        end


        frame(ani, mypl)
    end
    
    gif(ani, fps=15)
end